# ML Lab 01: Train Your First Model

You've heard "machine learning model" a thousand times. But what actually *is* a model?

In this lab, you'll find out by building one from scratch. You'll train a text classifier,
discover why accuracy is a trap, learn what overfitting looks like, and save your model to
a file you can inspect. By the end, there's no more mystery — a model is just learned
parameters stored on disk.

---
## Section 1: Load and Explore the Data

We'll use the **20 Newsgroups** dataset, which is built into scikit-learn (no downloads needed).
It contains ~20,000 newsgroup posts across 20 topics. We'll pick two for binary classification:
- `rec.sport.baseball` (baseball discussions)
- `sci.space` (space science discussions)

Our goal: given a text post, predict which topic it belongs to.

In [ ]:
from sklearn.datasets import fetch_20newsgroups

categories = ['rec.sport.baseball', 'sci.space']

train_data = fetch_20newsgroups(
    subset='train',
    categories=categories,
    shuffle=True,
    random_state=42
)

test_data = fetch_20newsgroups(
    subset='test',
    categories=categories,
    shuffle=True,
    random_state=42
)

print(f"Training samples: {len(train_data.data)}")
print(f"Test samples:     {len(test_data.data)}")
print(f"Classes:          {train_data.target_names}")
print(f"Label encoding:   0 = {train_data.target_names[0]}, 1 = {train_data.target_names[1]}")

In [ ]:
# Let's look at one example from each class
import numpy as np

for label in [0, 1]:
    idx = np.where(train_data.target == label)[0][0]
    print(f"=== Class: {train_data.target_names[label]} ===")
    print(train_data.data[idx][:500])
    print("...\n")

In [ ]:
# Check class balance
import pandas as pd

train_counts = pd.Series(train_data.target).value_counts().sort_index()
print("Training set class distribution:")
for idx, count in train_counts.items():
    print(f"  {train_data.target_names[idx]}: {count} samples ({count/len(train_data.target)*100:.1f}%)")

print(f"\nRoughly balanced? {'Yes' if abs(train_counts.iloc[0] - train_counts.iloc[1]) / len(train_data.target) < 0.1 else 'No'}")

**What you should see:** Two roughly balanced classes with ~600 samples each. The text is messy real-world data — email headers, signatures, quoted replies. This is typical of real ML data.

---

## Section 2: Your First Model

To classify text, we need to convert words into numbers. We'll use **TF-IDF** (Term Frequency-Inverse Document Frequency), which scores each word by:
- How often it appears in *this* document (TF — more = higher)
- How rare it is across *all* documents (IDF — rarer = higher)

Then we'll feed those numbers into **Logistic Regression**, a simple but powerful classifier.

scikit-learn's `Pipeline` chains these two steps together so we can treat them as one model.

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Build the pipeline: text -> TF-IDF vectors -> Logistic Regression
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(max_features=10000, stop_words='english')),
    ('clf', LogisticRegression(max_iter=1000, random_state=42))
])

# Train on training data
pipeline.fit(train_data.data, train_data.target)

# Predict on test data
predictions = pipeline.predict(test_data.data)

accuracy = accuracy_score(test_data.target, predictions)
print(f"Accuracy: {accuracy:.3f}")
print(f"\nThat's {accuracy*100:.1f}% correct! Looks great... right?")

**Feels good, doesn't it?** 96%+ accuracy on your first model! But before you celebrate, let's see why accuracy can lie to you.

---

## Section 3: Why Accuracy Lies

Imagine a dataset where 95% of samples are class A and 5% are class B.
A model that *always* predicts class A would score 95% accuracy — without learning anything.

Let's prove it.

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

# Create an imbalanced dataset: 95% baseball, 5% space
np.random.seed(42)

# Get all baseball (class 0) and space (class 1) samples
baseball_idx = np.where(train_data.target == 0)[0]
space_idx = np.where(train_data.target == 1)[0]

# Take 950 baseball samples and 50 space samples
n_majority = 475
n_minority = 25
imb_idx = np.concatenate([
    baseball_idx[:n_majority],
    space_idx[:n_minority]
])
np.random.shuffle(imb_idx)

imb_texts = [train_data.data[i] for i in imb_idx]
imb_labels = train_data.target[imb_idx]

print(f"Imbalanced dataset: {sum(imb_labels == 0)} baseball, {sum(imb_labels == 1)} space")
print(f"Majority class is {sum(imb_labels == 0)/len(imb_labels)*100:.1f}% of the data")

In [ ]:
# Train on the imbalanced data
imb_pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(max_features=10000, stop_words='english')),
    ('clf', LogisticRegression(max_iter=1000, random_state=42))
])

imb_pipeline.fit(imb_texts, imb_labels)
imb_predictions = imb_pipeline.predict(test_data.data)

imb_accuracy = accuracy_score(test_data.target, imb_predictions)
print(f"Accuracy: {imb_accuracy:.3f}")
print(f"\nStill looks decent... but let's look deeper.")

In [ ]:
# The confusion matrix reveals the truth
cm = confusion_matrix(test_data.target, imb_predictions)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Confusion matrix heatmap
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=train_data.target_names,
            yticklabels=train_data.target_names,
            ax=axes[0])
axes[0].set_xlabel('Predicted')
axes[0].set_ylabel('Actual')
axes[0].set_title('Confusion Matrix (Imbalanced Training)')

# Compare with balanced model
cm_balanced = confusion_matrix(test_data.target, predictions)
sns.heatmap(cm_balanced, annot=True, fmt='d', cmap='Greens',
            xticklabels=train_data.target_names,
            yticklabels=train_data.target_names,
            ax=axes[1])
axes[1].set_xlabel('Predicted')
axes[1].set_ylabel('Actual')
axes[1].set_title('Confusion Matrix (Balanced Training)')

plt.tight_layout()
plt.show()

In [ ]:
# The full classification report tells the real story
print("=== Imbalanced Model ===")
print(classification_report(test_data.target, imb_predictions,
                            target_names=train_data.target_names))

print("\n=== Balanced Model ===")
print(classification_report(test_data.target, predictions,
                            target_names=train_data.target_names))

**Key insight:** The imbalanced model has high accuracy but terrible **recall** for the minority class. The confusion matrix shows it barely identifies space posts correctly.

**Metrics to know:**
- **Precision**: Of all the samples predicted as class X, how many actually were? ("When I say space, am I right?")
- **Recall**: Of all actual class X samples, how many did I find? ("Did I find all the space posts?")
- **F1**: Harmonic mean of precision and recall — one number that balances both

**Rule of thumb:** If your classes are imbalanced, *never* trust accuracy alone. Always check the confusion matrix.

---

## Section 4: Proper Train/Test/Validation Split

So far we've used two sets: train and test. But there's a subtle problem.

When you tune your model (change parameters, try different algorithms) based on test set results,
you're *leaking information* from the test set into your decisions. Your test set is no longer truly "unseen."

The fix: **three-way split**.

```
┌─────────────────────────────────────────────────────────────┐
│                     All Data                                │
├────────────────────────┬─────────────┬──────────────────────┤
│    Training (60%)      │  Val (20%)  │    Test (20%)        │
│    Learn patterns      │  Tune here  │    Final eval only   │
└────────────────────────┴─────────────┴──────────────────────┘
```

- **Train:** Model learns from this
- **Validation:** You use this to compare models and tune parameters
- **Test:** Touch this ONCE at the very end for your final score

In [ ]:
from sklearn.model_selection import train_test_split

# Combine all data first, then split three ways
all_texts = train_data.data + test_data.data
all_labels = np.concatenate([train_data.target, test_data.target])

# 60% train, 20% validation, 20% test
X_train, X_temp, y_train, y_temp = train_test_split(
    all_texts, all_labels, test_size=0.4, random_state=42, stratify=all_labels
)

X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp
)

print(f"Train:      {len(X_train)} samples ({len(X_train)/len(all_texts)*100:.0f}%)")
print(f"Validation: {len(X_val)} samples ({len(X_val)/len(all_texts)*100:.0f}%)")
print(f"Test:       {len(X_test)} samples ({len(X_test)/len(all_texts)*100:.0f}%)")

In [ ]:
# The proper workflow: train, validate, then test once

# Step 1: Train the model
model = Pipeline([
    ('tfidf', TfidfVectorizer(max_features=10000, stop_words='english')),
    ('clf', LogisticRegression(max_iter=1000, random_state=42))
])
model.fit(X_train, y_train)

# Step 2: Evaluate on validation set (use this to tune)
val_accuracy = model.score(X_val, y_val)
print(f"Validation accuracy: {val_accuracy:.3f}")
print("(Use this to decide if you need to change your approach)")

# Step 3: Final evaluation on test set (do this ONCE)
test_accuracy = model.score(X_test, y_test)
print(f"\nFinal test accuracy: {test_accuracy:.3f}")
print("(This is your real-world performance estimate)")

**Why does this matter?** In practice, if you check your test score after every change, you'll unconsciously optimize for the test set. The validation set gives you a "safe" set to iterate on without contaminating your final evaluation.

---

## Section 5: Overfit on Purpose

**Overfitting** is when a model memorizes the training data instead of learning general patterns.
It gets perfect scores on training data but fails on new data.

Let's make this happen intentionally with a Decision Tree that has no limits.

In [ ]:
from sklearn.tree import DecisionTreeClassifier

# An unconstrained Decision Tree will memorize everything
overfit_model = Pipeline([
    ('tfidf', TfidfVectorizer(max_features=10000, stop_words='english')),
    ('clf', DecisionTreeClassifier(random_state=42))  # No max_depth — it will grow until it memorizes
])

overfit_model.fit(X_train, y_train)

train_acc = overfit_model.score(X_train, y_train)
val_acc = overfit_model.score(X_val, y_val)
test_acc = overfit_model.score(X_test, y_test)

print(f"Train accuracy:      {train_acc:.3f}")
print(f"Validation accuracy: {val_acc:.3f}")
print(f"Test accuracy:       {test_acc:.3f}")
print(f"\nTrain-Test gap:      {train_acc - test_acc:.3f}")
print(f"\n{'🚨 OVERFITTING!' if train_acc - test_acc > 0.1 else 'Looks OK'}")
print("The model memorized the training data — 100% on train, but much worse on new data.")

In [ ]:
# Fix it: constrain the tree
depths = [1, 3, 5, 10, 20, 50, None]  # None = no limit
train_scores = []
val_scores = []

for depth in depths:
    dt = Pipeline([
        ('tfidf', TfidfVectorizer(max_features=10000, stop_words='english')),
        ('clf', DecisionTreeClassifier(max_depth=depth, random_state=42))
    ])
    dt.fit(X_train, y_train)
    train_scores.append(dt.score(X_train, y_train))
    val_scores.append(dt.score(X_val, y_val))

# Visualize the overfitting curve
fig, ax = plt.subplots(figsize=(10, 6))
x_labels = [str(d) if d else 'None' for d in depths]
x_pos = range(len(depths))

ax.plot(x_pos, train_scores, 'o-', label='Train Accuracy', linewidth=2, markersize=8)
ax.plot(x_pos, val_scores, 's-', label='Validation Accuracy', linewidth=2, markersize=8)
ax.set_xticks(x_pos)
ax.set_xticklabels(x_labels)
ax.set_xlabel('max_depth')
ax.set_ylabel('Accuracy')
ax.set_title('Overfitting: Train vs Validation Accuracy by Tree Depth')
ax.legend()
ax.grid(True, alpha=0.3)

# Highlight the gap
best_val_idx = np.argmax(val_scores)
ax.axvline(x=best_val_idx, color='green', linestyle='--', alpha=0.5,
           label=f'Best validation (depth={x_labels[best_val_idx]})')
ax.legend()

plt.tight_layout()
plt.show()

print(f"\nBest validation accuracy: {max(val_scores):.3f} at max_depth={depths[best_val_idx]}")
print(f"Unconstrained tree: train={train_scores[-1]:.3f}, val={val_scores[-1]:.3f}")
print(f"\nThe gap between train and validation curves IS overfitting.")

**Key insight:** As model complexity increases (deeper tree), training accuracy goes up but validation accuracy eventually goes down. The sweet spot is where the validation accuracy peaks.

Compare this to Logistic Regression, which is simpler and doesn't overfit as easily:

In [ ]:
# Logistic Regression doesn't overfit here because it's a simpler model
lr_train = model.score(X_train, y_train)
lr_val = model.score(X_val, y_val)

print(f"Logistic Regression:")
print(f"  Train accuracy:      {lr_train:.3f}")
print(f"  Validation accuracy: {lr_val:.3f}")
print(f"  Gap:                 {lr_train - lr_val:.3f}")
print(f"\nMuch smaller gap = less overfitting. Simpler models generalize better.")

---
## Section 6: Save and Inspect the Model

A trained model is just a collection of **learned parameters** (coefficients, thresholds, vocabulary mappings).
We can save it to a file, load it later, and inspect what's inside.

This file IS your model. Everything the model learned lives here.

In [ ]:
import joblib
import os

# Save the trained pipeline (TF-IDF + Logistic Regression)
model_path = 'sentiment_model.joblib'
joblib.dump(model, model_path)

size_bytes = os.path.getsize(model_path)
size_mb = size_bytes / (1024 * 1024)
print(f"Model saved to: {model_path}")
print(f"File size: {size_mb:.2f} MB ({size_bytes:,} bytes)")
print(f"\nThis file contains everything the model learned:")
print(f"  - The TF-IDF vocabulary ({len(model.named_steps['tfidf'].vocabulary_):,} words)")
print(f"  - The IDF weights for each word")
print(f"  - The logistic regression coefficients")
print(f"  - The intercept (bias) term")

In [ ]:
# Load it back and verify it works
loaded_model = joblib.load(model_path)

# Test on a few examples
test_texts = [
    "The pitcher threw a fastball and struck out the batter in the ninth inning.",
    "NASA launched a new satellite to study the atmosphere of Mars.",
    "The home run in the bottom of the seventh sealed the championship.",
    "The telescope captured images of a distant galaxy cluster."
]

loaded_predictions = loaded_model.predict(test_texts)
loaded_probabilities = loaded_model.predict_proba(test_texts)

print("Predictions from the loaded model:\n")
for text, pred, proba in zip(test_texts, loaded_predictions, loaded_probabilities):
    label = train_data.target_names[pred]
    confidence = max(proba) * 100
    print(f"  [{label:>20s}] ({confidence:.0f}% confident) \"{text[:60]}...\"")

In [ ]:
# Inspect the learned parameters: what words matter most?
feature_names = model.named_steps['tfidf'].get_feature_names_out()
coefficients = model.named_steps['clf'].coef_[0]

# Sort by coefficient value
sorted_idx = np.argsort(coefficients)

n_top = 15

fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Top words for class 0 (baseball) — most negative coefficients
top_baseball = sorted_idx[:n_top]
axes[0].barh(range(n_top), coefficients[top_baseball], color='#2196F3')
axes[0].set_yticks(range(n_top))
axes[0].set_yticklabels([feature_names[i] for i in top_baseball])
axes[0].set_title(f'Top {n_top} words → {train_data.target_names[0]}')
axes[0].set_xlabel('Coefficient (more negative = stronger signal)')

# Top words for class 1 (space) — most positive coefficients
top_space = sorted_idx[-n_top:]
axes[1].barh(range(n_top), coefficients[top_space], color='#FF9800')
axes[1].set_yticks(range(n_top))
axes[1].set_yticklabels([feature_names[i] for i in top_space])
axes[1].set_title(f'Top {n_top} words → {train_data.target_names[1]}')
axes[1].set_xlabel('Coefficient (more positive = stronger signal)')

plt.suptitle('What the Model Learned: Most Important Words per Class', fontsize=14)
plt.tight_layout()
plt.show()

print(f"\nThe model learned {len(feature_names):,} word-to-number mappings.")
print(f"Each word has a coefficient: positive = space, negative = baseball.")
print(f"These coefficients ARE the model. That's all there is to it.")

In [ ]:
# Clean up the saved model file
os.remove(model_path)
print(f"Cleaned up {model_path}")

---
## Summary

You just built a machine learning model from scratch. Here's what you now know:

| Concept | What You Learned |
|---------|------------------|
| **A model** | Learned parameters (coefficients, weights) stored in a file |
| **Training** | Feeding data through an algorithm to learn those parameters |
| **Accuracy trap** | On imbalanced data, accuracy lies — use precision, recall, F1, confusion matrix |
| **Train/val/test** | Three-way split prevents you from fooling yourself during tuning |
| **Overfitting** | Memorizing training data (100% train, bad test) — fix with simpler models or constraints |
| **Model file** | A serialized object containing vocabulary + learned weights — nothing magical |

### What's Next?

In **ML Lab 02**, you'll take this saved model and deploy it behind a FastAPI endpoint — turning a file on disk into a live prediction service.